In [0]:
from google.colab import auth
auth.authenticate_user()

In [57]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmp8_4avjs9/pubring.gpg' created
gpg: /tmp/tmp8_4avjs9/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [58]:
!mkdir -p my_drive
!google-drive-ocamlfuse my_drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import numpy
import os
import pandas
import keras.optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.pipeline import Pipeline
from scipy import stats

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [0]:
base = 'my_drive'+os.sep+'ML_Project'+os.sep

In [0]:
X = pandas.read_csv(base+"X_train.txt", delim_whitespace=True, header=None)
Y = pandas.read_csv(base+"Y_train.txt", delim_whitespace=True, header=None)

In [0]:
X_main = X.values
Y_main = Y.values
X = X.sample(n=10000,replace="False")
Y = Y.sample(n=10000,replace="False")

In [0]:
X = X.values
Y = Y.values

In [0]:
sgd = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
adagrad = keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
adadelta = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
adamax = keras.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
nadam = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
sgd.__name__ = 'sgd'
rmsprop.__name__ = 'rmsprop'
adagrad.__name__ = 'adagrad'
adadelta.__name__ = 'adadelta'
adam.__name__ = 'adam'
adamax.__name__ = 'adamax'
nadam.__name__ = 'nadam'

In [0]:
standard_scaler = StandardScaler()
robust_scaler = RobustScaler(quantile_range=(25, 75))
quantile_uniform = QuantileTransformer(output_distribution='uniform')
quantile_gaussian = QuantileTransformer(output_distribution='normal')
min_max = MinMaxScaler(copy=True, feature_range=(0, 1))
standard_scaler.__name__ = 'standard_scaler'
robust_scaler.__name__ = 'robust_scaler'
quantile_uniform.__name__ = 'quantile_uniform'
quantile_gaussian.__name__ = 'quantile_gaussian'
min_max.__name__ = 'min_max'

In [0]:
standardizers = [standard_scaler,robust_scaler,quantile_uniform,quantile_gaussian]
#standardizers_1 = [standard_scaler,robust_scaler]
#standardizers = list(set(standardizers)-set(standardizers_1))
standardizers = [standard_scaler]
batch_size_list = [50,100]
batch_size_list = [50]
epoch_list = [i//10 for i in batch_size_list]
epoch_list = [10]
activators = ['softmax','softplus','softsign','linear','tanh','sigmoid','hard_sigmoid','elu','selu','relu']
#activators_1 = ['softmax','sigmoid','relu']
#activators = list(set(activators)-set(activators_1))
activators = ['sigmoid']
optimizers = [sgd,rmsprop,adagrad,adadelta,adam,adamax,nadam]
#optimizers_1 = [sgd,rmsprop,adam]
#optimizers = list(set(optimizers)-set(optimizers_1)) 
optimizers = [rmsprop]
estimators = dict()
kfold = KFold(n_splits=5, random_state=seed)
c = 0
for batch in batch_size_list:
  for epoch in epoch_list:
    for standard in standardizers:
      for opti in optimizers:
        for activator in activators:
          key = (batch,epoch,standard.__name__,opti.__name__,activator)
          c += 1
          if c%100==0:print(c, end=' ')
          def final_model():
            model = Sequential()
            model.add(Dense(30, input_dim=14, kernel_initializer='normal', activation=activator))
            model.add(Dense(5, kernel_initializer='normal', activation=activator))
#            model.add(Dense(20, input_dim=14, kernel_initializer='normal', activation=activator))
#            model.add(Dense(6, kernel_initializer='normal', activation=activator))
            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer=opti)
            return model
          estimator = []
          estimator.append(('standardize', standard))
          regressor = KerasRegressor(build_fn=final_model, epochs=epoch, batch_size=batch, verbose=0)
          estimator.append(('mlp', regressor))
          estimators[key]=estimator

In [64]:
len(estimators)

1

In [0]:
grid_search_results = dict()
grid_search_failure = dict()
alloc_failure = dict()

In [0]:
kfold = KFold(n_splits=3, random_state=seed)

In [60]:
for key in estimators:
  pipeline = Pipeline(estimators[key])
  print(key,end='\t')
  try:
    results = cross_val_score(pipeline, X, Y, cv=kfold)
    try:
#      grid_search_results[key] = (results, pipeline.fit(X_main,Y_main))
      grid_search_results[key] = (results, 'j')
      print(results.mean(),results.std())
    except Exception as a:
      print('Error X : ', a)
      alloc_failure[key] = a
  except Exception as a:
    print('Error Y : ', a, pipeline)
    grid_search_failure[key] = a
  print('*'*80)

(50, 10, 'standard_scaler', 'rmsprop', 'softsign')	-3.532128057878056 0.1543854508441383
********************************************************************************
(50, 10, 'standard_scaler', 'rmsprop', 'tanh')	-3.5346862249554647 0.1561008989323906
********************************************************************************
(50, 10, 'standard_scaler', 'rmsprop', 'elu')	-3.53451703183195 0.1589440737216339
********************************************************************************
(50, 10, 'standard_scaler', 'rmsprop', 'linear')	-3.529105872528365 0.15328707760861585
********************************************************************************
(50, 10, 'standard_scaler', 'rmsprop', 'selu')	-3.5294833740933718 0.15223803142651915
********************************************************************************
(50, 10, 'standard_scaler', 'rmsprop', 'softplus')	-3.537334238226162 0.15713728139211872
************************************************************************

In [0]:
import pickle

In [0]:
grid_search_output = []
for key in grid_search_results:
  vals = [i for i in key]
  res = grid_search_results[key][0]
#  vals.append(res)
  vals.append(res.mean())
  vals.append(res.std())
  vals.append(-1*res.mean()-res.std())
  vals.append(-1*res.mean()+res.std())
  grid_search_output.append(vals)

In [62]:
for i in grid_search_output: print(i)

[50, 10, 'standard_scaler', 'rmsprop', 'softsign', -3.532128057878056, 0.1543854508441383, 3.3777426070339174, 3.6865135087221943]
[50, 10, 'standard_scaler', 'rmsprop', 'tanh', -3.5346862249554647, 0.1561008989323906, 3.378585326023074, 3.6907871238878553]
[50, 10, 'standard_scaler', 'rmsprop', 'elu', -3.53451703183195, 0.1589440737216339, 3.375572958110316, 3.693461105553584]
[50, 10, 'standard_scaler', 'rmsprop', 'linear', -3.529105872528365, 0.15328707760861585, 3.3758187949197493, 3.6823929501369808]
[50, 10, 'standard_scaler', 'rmsprop', 'selu', -3.5294833740933718, 0.15223803142651915, 3.3772453426668525, 3.681721405519891]
[50, 10, 'standard_scaler', 'rmsprop', 'softplus', -3.537334238226162, 0.15713728139211872, 3.3801969568340433, 3.6944715196182805]
[50, 10, 'standard_scaler', 'rmsprop', 'hard_sigmoid', -3.53652557836152, 0.15311945126600152, 3.3834061270955185, 3.689645029627522]


In [0]:
with open(base+'nn_1.pkl', 'wb') as output:
    pickle.dump(grid_search_output, output, pickle.HIGHEST_PROTOCOL)

In [0]:
kfold = KFold(n_splits=5, random_state=seed)

In [79]:
pipeline = estimators[(50,10,standard_scaler.__name__,rmsprop.__name__,'sigmoid')]
pipeline = Pipeline(pipeline)
result = cross_val_score(pipeline, X, Y, cv=kfold)
pipeline.fit(X_main,Y_main)
print(result.mean(),result.std())

-3.50891986650601 0.28643941866367034


In [85]:
pipeline = estimators[(50,10,standard_scaler.__name__,rmsprop.__name__,'sigmoid')]
pipeline = Pipeline(pipeline)
result = cross_val_score(pipeline, X, Y, cv=kfold)
pipeline.fit(X_main,Y_main)
print(result.mean(),result.std())

-3.508146385975182 0.28859346561558546


In [0]:
adam = keras.optimizers.Adam(lr=0.02, beta_1=0.8, beta_2=0.699, epsilon=None, decay=0.0, amsgrad=False)
rmsprop = keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
sgd = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
def final_model():
  model = Sequential()
  model.add(Dense(20, input_dim=14, kernel_initializer='normal', activation='relu'))
  model.add(Dense(6, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  model.compile(loss='mean_squared_error', optimizer=sgd)
  return model
estimator = []
estimator.append(('standardize', min_max))
regressor = KerasRegressor(build_fn=final_model, epochs=20, batch_size=50, verbose=0)
estimator.append(('mlp', regressor))
pip = Pipeline(estimator)

In [0]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pip, X, Y, cv=kfold, verbose=2)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

[CV]  ................................................................
[CV] ................................................. , total= 4.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.4min remaining:    0.0s


[CV] ................................................. , total= 6.4min
[CV]  ................................................................
[CV] ................................................. , total= 7.8min
[CV]  ................................................................
[CV] ................................................. , total= 8.1min
[CV]  ................................................................


In [0]:
# evaluate model with standardized dataset
X_test = pandas.read_csv(base+"X_test.txt", delim_whitespace=True, header=None)
X_test = X_test.values
prediction = pipeline.predict(X_test)

In [0]:
fh = open(base+'predictions_nn.csv','w')
fh.write('ID,Prediction\n')
for i,yi in enumerate(prediction):
    fh.write('{},{}\n'.format(i,yi))
fh.close()

In [88]:
unique, counts = numpy.unique(prediction, return_counts=True)
print (numpy.asarray((unique, counts)).T)

[[0.08126303 1.        ]
 [0.08647559 1.        ]
 [0.08683569 1.        ]
 ...
 [3.07366085 1.        ]
 [3.09059834 1.        ]
 [3.10574508 1.        ]]
